# TM prediction with Prime

This tutorial demonstrates how to predict the OGT of a protein using a pretrained model from the Prime model.

We provide:

- The sequences, a FASTA file.

Goals
Obtain an predicted TM for each sequence.


## Import the necessary libraries and modules.

In [11]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
from Bio import SeqIO
from tqdm.notebook import tqdm

## Prepare data path

In [12]:
sequence_file = "example.fasta"

## Load model and Tokenizer

In [15]:
model_path = "AI4Protein/Prime_690M_TM"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model.eval()
model = model.to(device)

## Prediction

In [17]:
tm = []
with torch.no_grad():
    for record in tqdm(list(SeqIO.parse(sequence_file, "fasta"))):
        sequence = str(record.seq)
        tokenied_results = tokenizer(sequence, return_tensors="pt")
        input_ids = tokenied_results.input_ids.to(device)
        attention_mask = tokenied_results.attention_mask.to(device)
        logits = model(input_ids=input_ids, attention_mask=attention_mask).predicted_values
        tm.append(logits.item())

  0%|          | 0/14 [00:00<?, ?it/s]